In [9]:
import pandas as pd
rep=pd.read_csv('RecipeNLG_dataset.csv')
rep.head(10)

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."
5,5,Cheeseburger Potato Soup,"[""6 baking potatoes"", ""1 lb. of extra lean gro...","[""Wash potatoes; prick several times with a fo...",www.cookbooks.com/Recipe-Details.aspx?id=20115,Gathered,"[""baking potatoes"", ""extra lean ground beef"", ..."
6,6,Rhubarb Coffee Cake,"[""1 1/2 c. sugar"", ""1/2 c. butter"", ""1 egg"", ""...","[""Cream sugar and butter."", ""Add egg and beat ...",www.cookbooks.com/Recipe-Details.aspx?id=210288,Gathered,"[""sugar"", ""butter"", ""egg"", ""buttermilk"", ""flou..."
7,7,Scalloped Corn,"[""1 can cream-style corn"", ""1 can whole kernel...","[""Mix together both cans of corn, crackers, eg...",www.cookbooks.com/Recipe-Details.aspx?id=876969,Gathered,"[""cream-style corn"", ""whole kernel corn"", ""cra..."
8,8,Nolan'S Pepper Steak,"[""1 1/2 lb. round steak (1-inch thick), cut in...","[""Roll steak strips in flour."", ""Brown in skil...",www.cookbooks.com/Recipe-Details.aspx?id=375254,Gathered,"[""tomatoes"", ""water"", ""onions"", ""Worcestershir..."
9,9,Millionaire Pie,"[""1 large container Cool Whip"", ""1 large can c...","[""Empty Cool Whip into a bowl."", ""Drain juice ...",www.cookbooks.com/Recipe-Details.aspx?id=794547,Gathered,"[""pineapple"", ""condensed milk"", ""lemons"", ""pec..."


In [13]:
rep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2231142 entries, 0 to 2231141
Data columns (total 7 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   Unnamed: 0   int64 
 1   title        object
 2   ingredients  object
 3   directions   object
 4   link         object
 5   source       object
 6   NER          object
dtypes: int64(1), object(6)
memory usage: 119.2+ MB


In [19]:
import pandas as pd
import re
from ast import literal_eval

def clean_ingredient(text):
    text = text.lower()

    # Remove numbers, fractions, units, and cooking descriptors
    remove_words = [
        "cup", "cups", "tablespoon", "tablespoons", "tbsp",
        "teaspoon", "teaspoons", "tsp", "grams", "g", "kg", "ml", "l",
        "oz", "pound", "pinch", "clove", "slice", "can", "package",
        "chopped", "diced", "minced", "fresh", "ground", "large", "small", "of", "to taste"
    ]
    pattern = r'\b(?:' + '|'.join(remove_words) + r')\b'
    text = re.sub(pattern, '', text)

    text = re.sub(r'\b\d+[\d\s\/\.⁄¼½¾⅓⅔⅛⅜⅝⅞]*\b', '', text)  # remove quantities
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra spaces

    return text

def extract_clean_ingredients(df_column):
    cleaned_ingredients = set()

    for row in df_column:
        try:
            items = literal_eval(row)
            for item in items:
                cleaned = clean_ingredient(item)
                if cleaned:
                    cleaned_ingredients.add(cleaned)
        except:
            continue

    return sorted(list(cleaned_ingredients))
ingredient_list = extract_clean_ingredients(rep['ingredients'])
print(ingredient_list[:30])  # Preview first 30 clean ingredients
print(f"Total unique: {len(ingredient_list)}")


['a', 'a a balsamic vinegar', 'a a carrots coarsely grated', 'a a cayenne pepper', 'a a club house onion powder', 'a a few drops lemon juice or a nip cream tartar optional', 'a a frozen shelled edamame thawed', 'a a liquid honey', 'a a mccormick gourmet caraway seed', 'a a mccormick gourmet coarse grind black pepper', 'a a ml apple cider vinegar', 'a a nutmeg grated', 'a a peanut butter', 'a a quinoa', 'a a red onion', 'a a salt', 'a a salt and pepper', 'a a soy sauce', 'a a sprouts your favourite variety', 'a a sugar', 'a a tomato paste', 'a a unsweetened cocoa powder', 'a aallspice and nutmeg', 'a aarborio or italian style rice', 'a about fleur de sel or other flakey sea salt maldon works fine here', 'a about parmesan rinds', 'a about saffron threads', 'a abunch coriander', 'a abunch flat parsley', 'a aburaage']
Total unique: 2505971


In [21]:
import re

def clean_raw_ingredient(ing):
    ing = ing.lower().strip()

    # Remove leading "a", "a a", "a abunch", "a about", etc.
    ing = re.sub(r'^a( +[a-z]+)*', '', ing)

    # Remove known optional phrases
    ing = re.sub(r'\b(optional|your favourite variety|mccormick gourmet|club house)\b', '', ing)

    # Remove units and adjectives
    remove_words = [
        "few", "drops", "coarsely", "grated", "ground", "fine", "unsweetened",
        "liquid", "ml", "frozen", "shelled", "thawed", "nip", "optional", "style", "other"
    ]
    pattern = r'\b(?:' + '|'.join(remove_words) + r')\b'
    ing = re.sub(pattern, '', ing)

    # Remove numbers, extra spaces, punctuation
    ing = re.sub(r'\d+', '', ing)
    ing = re.sub(r'[^a-z\s]', '', ing)
    ing = re.sub(r'\s+', ' ', ing).strip()

    return ing

cleaned_ingredients = sorted(set(filter(None, [clean_raw_ingredient(i) for i in ingredient_list])))
print(cleaned_ingredients[:30])
print(f"Total unique: {len(cleaned_ingredients)}")

['a a cooked beet puree beets', 'a a half half', 'a aasticks butter softened to room temperature', 'a alpine pepper jus', 'a batch cornflake wreath dough or candy salad for serving', 'a bit brown sugar if the mango doesnt make it sweet enough it usually does', 'a bit freshly parmesan cheese', 'a candy thermometer', 'a cayenne pepper or chili', 'a chili pepper and sesame oil for smell and flavor', 'a chinese mushrooms grs makes it even tastier', 'a coconut aminos', 'a couple cloves crushed garlic', 'a couple pimento', 'a courgette flowers', 'a cumin', 'a dash campari', 'a dash red balsamic vinegar', 'a dozen cherry tomatoes quartered', 'a drizzle hazelnut oil or walnut oil for serving', 'a food coloring', 'a fragrant sesame oil', 'a garlic', 'a ginger', 'a good phillips seafood seasoning or old bay', 'a handful cilantro', 'a handful coriander leaves picked and', 'a handful jerusalem artichokes peeled and halved glb oz potatoes', 'a handful sunflower seeds', 'a handful watercress capers 

In [24]:
inv=pd.read_csv('home_inventory.csv')
#clean deplcated items
inv=inv.drop_duplicates(subset='name', keep='first')

In [25]:
# save
inv.to_csv("home_inventory.csv")